Fix metadata song

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [31]:
import re

# Function to clean file names
def clean_file_name(file_name):
    # Remove invalid characters from the file name
    cleaned_name = re.sub(r'[<>:"/\\|?*]', '', file_name)
    return cleaned_name

In [39]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from mutagen.id3 import ID3, TIT2, TPE1, TALB

# Set up the Spotipy client
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Function to update metadata
def update_metadata(file_path):
    # Extract song name from the file name
    song_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Search for the song using the Spotify API
    results = sp.search(q=song_name, limit=1, type='track')
    
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        
        # Extract the relevant metadata
        title = track['name']
        artist = track['artists'][0]['name']
        album = track['album']['name']
        
        # Update the metadata of the MP3 file
        try:
            audio = ID3(file_path)
            audio['TIT2'] = TIT2(encoding=3, text=title)
            audio['TPE1'] = TPE1(encoding=3, text=artist)
            audio['TALB'] = TALB(encoding=3, text=album)
            audio.save()
        except:
            print(f"No results found for '{song_name}'.")
        
        # Generate the new file name
        new_file_name = f"{artist} - {title}.mp3"
        new_file_name = clean_file_name(new_file_name)  # Clean the new file name
        new_file_path = os.path.join(os.path.dirname(file_path), new_file_name)
        
        # Check if the new file path already exists
        counter = 1
        while os.path.exists(new_file_path):
            file_name, file_extension = os.path.splitext(new_file_name)
            new_file_name = f"{file_name} ({counter}){file_extension}"
            new_file_path = os.path.join(os.path.dirname(file_path), new_file_name)
            counter += 1

        # Rename the file
        os.rename(file_path, new_file_path)

        # Store the original and new file names
        with open('file_names_mapping.txt', 'a') as file:
            file.write(f"{file_name},{new_file_name}\n")

    else:
        print(f"No results found for '{song_name}'.")

# Specify the root directory where your MP3 files are located
root_directory = 'D:\Music'

# Recursively traverse the directory and update metadata and file names
for dirpath, dirnames, filenames in os.walk(root_directory):
    for filename in filenames:
        if filename.endswith('.mp3'):
            file_path = os.path.join(dirpath, filename)
            update_metadata(file_path)


No results found for 'The Script - Before the Worst'.
No results found for 'The Script - We Cry'.


In [40]:
# Undo the changes and revert the file names
def undo_changes():
    # Read the file_names_mapping.txt to get the original and new file names
    with open('file_names_mapping.txt', 'r') as file:
        mappings = file.readlines()

    # Iterate through the mappings and revert the file names
    for mapping in mappings:
        mapping = mapping.strip().split(',')
        original_file_name, new_file_name = mapping[0], mapping[1]
        original_file_path = os.path.join(root_directory, original_file_name)
        new_file_path = os.path.join(root_directory, new_file_name)

        # Rename the file back to the original file name
        os.rename(new_file_path, original_file_path)

    # Remove the file_names_mapping.txt file
    os.remove('file_names_mapping.txt')

In [10]:
import os

def print_directory_structure_with_vtt(directory):
    """
    Traverse the given directory, print the directory structure with headings,
    and print only the file names ending with ".en.vtt".
    
    Args:
    - directory (str): The directory path to traverse
    """
    def print_directory_recursively(root_dir, depth):
        """
        Recursive function to print the directory structure with headings
        
        Args:
        - root_dir (str): The root directory to start traversal from
        - depth (int): The depth of the current directory in the hierarchy
        """
        for item in sorted(os.listdir(root_dir)):
            item_path = os.path.join(root_dir, item)
            if os.path.isdir(item_path):
                print("#" * (depth) + " " + item)
                print_directory_recursively(item_path, depth + 1)
            elif item.endswith(".en.vtt"):
                print("# " * depth + "- " + item)

    print("# Directory Structure")
    print_directory_recursively(directory, 0)

# Example usage:
directory_path = "/home/al/Documents/courses/Udacity - Artificial Intelligence AI for Trading v1.0.0"
print_directory_structure_with_vtt(directory_path)

# Directory Structure
# Part 01-Module 01-Lesson 04_Stock Prices
# - 01. M1L2 01 Stocks V6-23sv5ey0ySs.en.vtt
# - 02. M1L2 01 Stock  Pt II V1-SGb54HLbk1g.en.vtt
# - 02. M1L2 02 Stock Prices V7-l_PilXVuh8I.en.vtt
# Part 01-Module 01-Lesson 05_Market Mechanics
# - 01. M1L3 01 Intro V4-LE-4Xf8lzHk.en.vtt
# - 02. M1L3 02 Farmers Market V3-i_itXOdetCc.en.vtt
# - 03. M1L3 03 Trading Stocks V3-GHoRtfUrUMc.en.vtt
# - 04. M1L3 04 Liquidity V4-KNVQeH6Y_YA.en.vtt
# - 05. M1L3 08 Tick Data V4-2O0eSKmI6YQ.en.vtt
# - 06. M1L3 09 Open High Low Close V4-FgNY4YgVWFk.en.vtt
# - 08. M1L3 10 Volume V3-DFp7kp0xRCo.en.vtt
# - 09. M1L3 12 Gaps In Market Data V3-jMT3VbUGiZI.en.vtt
# - 10. M1L3 14 Markets In Different Timezones V3-wmmEpPM-HVs.en.vtt
# - 11. M1L3 15 Outro V2-XVvfToYCsmo.en.vtt
# Part 01-Module 01-Lesson 06_Data Processing
# - 01. M1L4 01 Stock Data V2-sN0_IqmMGGA.en.vtt
# - 02. M1L4 02 Market Data V5-9aEp374GsgQ.en.vtt
# - 03. M1L4 04 Corporate Actions V5-S60WArbQK7k.en.vtt
# - 03. M1L4 04b Div

# Directory Structure
# Part 07-Module 01-Lesson 04_Decision Trees
  - 08. Entropy Formula-iZiSYrOKvpo.mp4
  - 08. Entropy Formula-iZiSYrOKvpo.pt-BR.vtt
  - 03. MLND SL DT 02 Recommending Apps 2 MAIN V3-KSrIYqKZwCA.en.vtt
  - 13. Information Gain-k9iZL53PAmw.en.vtt
  - 06. Student Admissions-TdgBi6LtOB8.mp4
  - 10. Entropy Formula-w73JTBVeyjE.mp4
  - 10. Entropy Formula-w73JTBVeyjE.zh-CN.vtt
  - 05. MLND SL DT 04 Q Student Admissions V3 MAIN V1-MOa335cQGI4.mp4
  - 04. Recommending Apps-nEvW8B1HNq4.en.vtt
  - 04. Recommending Apps 3.html
  - 07. Entropy-piLpj1V1HEk.mp4
## img
    - screen-shot-2018-05-22-at-12.27.22-pm.png
    - student-data.png
    - screen-shot-2018-01-06-at-9.30.27-pm.png
    - screen-shot-2018-01-06-at-8.13.20-pm.png
    - meme.png
    - trees.png
    - table.png
    - screen-shot-2018-05-22-at-12.25.34-pm.png
    - screen-shot-2018-01-06-at-9.41.01-pm.png
    - screen-shot-2018-05-22-at-12.27.55-pm.png
    - recommending-apps.png
  - 01. MLND SL DT 00 Intro V2-l34i

In [4]:
import os
import glob

def print_vtt_files_sorted(directory):
    """
    Traverse the given directory, sort the file names alphabetically,
    and print the file names of files ending with ".en.vtt".
    
    Args:
    - directory (str): The directory path to traverse
    """
    vtt_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".en.vtt"):
                vtt_files.append(os.path.join(root, file))
    
    vtt_files.sort()  # Sort the list of file names alphabetically
    
    for file_path in vtt_files:
        print(file_path)

# Example usage:
directory_path = 
print_vtt_files_sorted(directory_path)


/home/al/Documents/courses/Udacity - Artificial Intelligence AI for Trading v1.0.0/Part 01-Module 01-Lesson 04_Stock Prices/01. M1L2 01 Stocks V6-23sv5ey0ySs.en.vtt
/home/al/Documents/courses/Udacity - Artificial Intelligence AI for Trading v1.0.0/Part 01-Module 01-Lesson 04_Stock Prices/02. M1L2 01 Stock  Pt II V1-SGb54HLbk1g.en.vtt
/home/al/Documents/courses/Udacity - Artificial Intelligence AI for Trading v1.0.0/Part 01-Module 01-Lesson 04_Stock Prices/02. M1L2 02 Stock Prices V7-l_PilXVuh8I.en.vtt
/home/al/Documents/courses/Udacity - Artificial Intelligence AI for Trading v1.0.0/Part 01-Module 01-Lesson 05_Market Mechanics/01. M1L3 01 Intro V4-LE-4Xf8lzHk.en.vtt
/home/al/Documents/courses/Udacity - Artificial Intelligence AI for Trading v1.0.0/Part 01-Module 01-Lesson 05_Market Mechanics/02. M1L3 02 Farmers Market V3-i_itXOdetCc.en.vtt
/home/al/Documents/courses/Udacity - Artificial Intelligence AI for Trading v1.0.0/Part 01-Module 01-Lesson 05_Market Mechanics/03. M1L3 03 Trading 